# 🔥👊💨👤 Unified YOLO11n Model: Complete Petrol Pump Detection

> **Purpose**: Train ONE model for all petrol pump safety detections
> 
> **Target**: Jetson Orin Nano CCTV Monitoring
> 
> **Base Model**: YOLO11n (edge-optimized)

## Detection Classes:
| ID | Class | Priority | Use Case |
|---|-------|----------|----------|
| 0 | person | 🔴 High | Customer/employee tracking |
| 1 | fire | 🔴 Critical | Fuel ignition detection |
| 2 | smoke | 🔴 Critical | Early fire warning |
| 3 | violence | 🟡 Important | Security incidents |
| 4 | cigarette | 🔴 Critical | Ignition source at pump |

## Data Sources:
- **Fire**: FASDD Dataset (same as ProFSAM-Fire-Detector model)
- **Violence**: Roboflow datasets (same source as fight_detection_yolov8)
- **Person**: COCO person subset + custom person detection
- **Cigarette**: Roboflow smoking detection datasets

## Step 1: Environment Setup

In [ ]:
# Install required packages
!pip install ultralytics>=8.3.0 roboflow gdown kaggle -q

# Verify installation
import ultralytics
ultralytics.checks()
print(f"✅ Ultralytics version: {ultralytics.__version__}")

In [ ]:
import os
import shutil
import yaml
import json
from pathlib import Path
from roboflow import Roboflow
from ultralytics import YOLO

# Create working directories
BASE_DIR = Path('/content/unified_detection')
DATASETS_DIR = BASE_DIR / 'datasets'
MERGED_DIR = BASE_DIR / 'merged_dataset'
MODELS_DIR = BASE_DIR / 'trained_models'

for d in [DATASETS_DIR, MERGED_DIR, MODELS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f"✅ Working directory: {BASE_DIR}")

## Step 2: Configure Datasets

### Dataset Sources:
1. **FASDD (Fire & Smoke Detection Dataset)** - Used by ProFSAM-Fire-Detector
2. **Violence Detection** - Similar to Musawer14/fight_detection_yolov8 training data
3. **Person Detection** - COCO person subset + Roboflow person datasets
4. **Cigarette/Smoking Detection** - Roboflow smoking datasets

In [ ]:
# @title Configure Roboflow API Key { display-mode: "form" }
# Get your free API key from https://roboflow.com/
ROBOFLOW_API_KEY = "YOUR_API_KEY_HERE"  # @param {type:"string"}

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
print("✅ Roboflow configured")

In [ ]:
# ============================================
# TARGET UNIFIED CLASSES (5 classes total)
# ============================================
UNIFIED_CLASSES = ['person', 'fire', 'smoke', 'violence', 'cigarette']

# Dataset configurations - curated for petrol pump monitoring
# Sources aligned with HuggingFace models you referenced
DATASETS = {
    # Person Detection - critical for customer/employee tracking
    'person': {
        'workspace': 'roboflow-100',
        'project': 'people-detection-general',
        'version': 1,
        'class_mapping': {
            'person': 'person', 'Person': 'person', 
            'people': 'person', 'human': 'person'
        }
    },
    
    # Fire & Smoke - aligned with FASDD dataset used by ProFSAM-Fire-Detector
    'fire_smoke': {
        'workspace': 'fire-detection-kbsxn',
        'project': 'fire-detection-qagzv',
        'version': 2,
        'class_mapping': {
            'fire': 'fire', 'Fire': 'fire', 'flame': 'fire',
            'smoke': 'smoke', 'Smoke': 'smoke'
        }
    },
    
    # Additional fire dataset for better coverage
    'fire_smoke_2': {
        'workspace': 'fire-smoke-detection',
        'project': 'fire-and-smoke-xspvt',
        'version': 1,
        'class_mapping': {
            'fire': 'fire', 'Fire': 'fire',
            'smoke': 'smoke', 'Smoke': 'smoke'
        }
    },
    
    # Violence Detection - aligned with fight_detection_yolov8 training approach
    'violence': {
        'workspace': 'securityviolence',
        'project': 'violence-detection-bxcxf',
        'version': 1,
        'class_mapping': {
            'violence': 'violence', 'Violence': 'violence',
            'fight': 'violence', 'Fight': 'violence',
            'fighting': 'violence'
        }
    },
    
    # Additional violence dataset for better coverage
    'violence_2': {
        'workspace': 'jaishreeram-uqqfn',
        'project': 'violence_maksad',
        'version': 1,
        'class_mapping': {
            'violence': 'violence', 'Violence': 'violence'
        }
    },
    
    # Cigarette/Smoking Detection - critical for petrol pumps
    'smoking': {
        'workspace': 'smoker-detection',
        'project': 'smoker',
        'version': 1,
        'class_mapping': {
            'smoker': 'cigarette', 'smoking': 'cigarette',
            'cigarette': 'cigarette', 'Cigarette': 'cigarette'
        }
    }
}

print(f"🎯 Target classes: {UNIFIED_CLASSES}")
print(f"📦 Datasets to download: {len(DATASETS)}")
for name in DATASETS:
    print(f"   - {name}")

In [ ]:
def download_dataset(name, config):
    """Download a dataset from Roboflow"""
    try:
        print(f"\n📥 Downloading {name}...")
        project = rf.workspace(config['workspace']).project(config['project'])
        dataset = project.version(config['version']).download(
            "yolov8",
            location=str(DATASETS_DIR / name)
        )
        print(f"✅ Downloaded: {name}")
        return True
    except Exception as e:
        print(f"⚠️ Failed {name}: {e}")
        return False

# Download all datasets
print("="*50)
print("DOWNLOADING DATASETS")
print("="*50)
for name, config in DATASETS.items():
    download_dataset(name, config)

## Step 3: Download Additional Person Dataset (COCO subset)

In [ ]:
# Download COCO person subset for robust person detection
# This ensures we have high-quality person annotations

import urllib.request
import zipfile

def download_coco_person_subset():
    """Download a curated COCO person subset"""
    coco_person_dir = DATASETS_DIR / 'coco_person'
    
    # Try to get person detection from Roboflow
    try:
        print("\n📥 Downloading COCO Person subset...")
        project = rf.workspace("microsoft").project("coco")
        dataset = project.version("1").download(
            "yolov8",
            location=str(coco_person_dir)
        )
        print("✅ Downloaded COCO dataset")
        return True
    except:
        print("⚠️ COCO download failed, using Roboflow person dataset instead")
        return False

# Try to download, but don't fail if it doesn't work
# We already have person from 'roboflow-100/people-detection-general'
download_coco_person_subset()

## Step 4: Merge Datasets with Unified Class IDs

In [ ]:
def remap_labels(label_file, old_classes, class_mapping, unified_classes):
    """Remap class IDs in a YOLO label file to unified class IDs"""
    if not os.path.exists(label_file):
        return 0
    
    with open(label_file, 'r') as f:
        lines = f.readlines()
    
    new_lines = []
    remapped_count = 0
    
    for line in lines:
        parts = line.strip().split()
        if len(parts) >= 5:
            old_class_id = int(parts[0])
            if old_class_id < len(old_classes):
                old_class_name = old_classes[old_class_id]
                # Map to unified class name
                unified_name = class_mapping.get(old_class_name, None)
                if unified_name and unified_name in unified_classes:
                    new_class_id = unified_classes.index(unified_name)
                    parts[0] = str(new_class_id)
                    new_lines.append(' '.join(parts) + '\n')
                    remapped_count += 1
    
    with open(label_file, 'w') as f:
        f.writelines(new_lines)
    
    return remapped_count

def get_dataset_classes(dataset_path):
    """Read classes from dataset YAML"""
    yaml_file = dataset_path / 'data.yaml'
    if yaml_file.exists():
        with open(yaml_file, 'r') as f:
            data = yaml.safe_load(f)
            names = data.get('names', [])
            if isinstance(names, dict):
                return [names[i] for i in sorted(names.keys())]
            return names
    return []

print("✅ Utility functions ready")

In [ ]:
def merge_datasets():
    """Merge all downloaded datasets into a unified dataset"""
    
    # Create merged directory structure
    for split in ['train', 'valid', 'test']:
        (MERGED_DIR / split / 'images').mkdir(parents=True, exist_ok=True)
        (MERGED_DIR / split / 'labels').mkdir(parents=True, exist_ok=True)
    
    stats = {
        'total_images': {'train': 0, 'valid': 0, 'test': 0},
        'per_class': {cls: 0 for cls in UNIFIED_CLASSES},
        'per_dataset': {}
    }
    
    for dataset_name, config in DATASETS.items():
        dataset_path = DATASETS_DIR / dataset_name
        
        # Find the actual dataset directory (sometimes nested)
        for subdir in dataset_path.rglob('data.yaml'):
            dataset_path = subdir.parent
            break
        
        if not dataset_path.exists():
            print(f"⚠️ {dataset_name} not found, skipping...")
            continue
        
        # Get original classes
        old_classes = get_dataset_classes(dataset_path)
        print(f"\n📂 {dataset_name}: {old_classes}")
        
        dataset_count = 0
        
        for split in ['train', 'valid', 'test']:
            # Try different possible paths
            src_images = None
            for img_subdir in ['images', '']:
                test_path = dataset_path / split / img_subdir if img_subdir else dataset_path / split
                if test_path.exists():
                    imgs = list(test_path.glob('*.jpg')) + list(test_path.glob('*.png'))
                    if imgs:
                        src_images = test_path
                        break
            
            if not src_images:
                continue
            
            # Find labels directory
            label_dir = None
            for lbl_subdir in ['labels', '']:
                test_path = dataset_path / split / lbl_subdir if lbl_subdir else src_images.parent / 'labels'
                if test_path.exists():
                    label_dir = test_path
                    break
            
            # Copy images and remap labels
            for img_file in list(src_images.glob('*.jpg')) + list(src_images.glob('*.png')):
                # Create unique filename
                new_name = f"{dataset_name}_{img_file.name}"
                
                # Copy image
                dst_img = MERGED_DIR / split / 'images' / new_name
                shutil.copy(img_file, dst_img)
                
                # Copy and remap label
                if label_dir:
                    label_file = label_dir / (img_file.stem + '.txt')
                    if label_file.exists():
                        dst_label = MERGED_DIR / split / 'labels' / (new_name.rsplit('.', 1)[0] + '.txt')
                        shutil.copy(label_file, dst_label)
                        remap_labels(dst_label, old_classes, config['class_mapping'], UNIFIED_CLASSES)
                
                stats['total_images'][split] += 1
                dataset_count += 1
        
        stats['per_dataset'][dataset_name] = dataset_count
        print(f"   Added {dataset_count} images")
    
    # Create unified data.yaml
    data_yaml = {
        'path': str(MERGED_DIR),
        'train': 'train/images',
        'val': 'valid/images',
        'test': 'test/images',
        'names': {i: name for i, name in enumerate(UNIFIED_CLASSES)},
        'nc': len(UNIFIED_CLASSES)
    }
    
    with open(MERGED_DIR / 'data.yaml', 'w') as f:
        yaml.dump(data_yaml, f, default_flow_style=False)
    
    print(f"\n" + "="*50)
    print("MERGED DATASET SUMMARY")
    print("="*50)
    print(f"Train: {stats['total_images']['train']} images")
    print(f"Valid: {stats['total_images']['valid']} images")
    print(f"Test:  {stats['total_images']['test']} images")
    print(f"Total: {sum(stats['total_images'].values())} images")
    print(f"\nClasses: {UNIFIED_CLASSES}")
    print(f"\nPer-dataset breakdown:")
    for ds, count in stats['per_dataset'].items():
        print(f"   {ds}: {count} images")
    
    return MERGED_DIR / 'data.yaml'

# Merge all datasets
data_yaml_path = merge_datasets()

## Step 5: Train Unified YOLO11n Model

In [ ]:
# @title Training Configuration { display-mode: "form" }

# ============================================
# 🎯 YOLO11n - Recommended for Jetson Orin Nano
# ============================================
MODEL_SIZE = "yolo11n"  # @param ["yolo11n", "yolo11s", "yolov8n", "yolov8s"]
EPOCHS = 100  # @param {type:"slider", min:50, max:300, step:10}
IMAGE_SIZE = 640  # @param [416, 512, 640, 800]
BATCH_SIZE = 16  # @param [8, 16, 32]
PATIENCE = 25  # @param {type:"slider", min:10, max:50, step:5}

print("\n" + "="*50)
print("TRAINING CONFIGURATION")
print("="*50)
print(f"Model: {MODEL_SIZE}")
print(f"Classes: {UNIFIED_CLASSES}")
print(f"Epochs: {EPOCHS}")
print(f"Image Size: {IMAGE_SIZE}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Early Stop Patience: {PATIENCE}")

In [ ]:
# Load pre-trained YOLO11n model
print(f"\n📦 Loading {MODEL_SIZE}...")
model = YOLO(f'{MODEL_SIZE}.pt')

# Train on merged dataset
# Settings aligned with ProFSAM-Fire-Detector training approach
print(f"\n🚀 Starting training...")
print(f"   This will take 1-3 hours depending on dataset size")

results = model.train(
    data=str(data_yaml_path),
    epochs=EPOCHS,
    imgsz=IMAGE_SIZE,
    batch=BATCH_SIZE,
    patience=PATIENCE,
    device=0,
    workers=4,
    project=str(MODELS_DIR),
    name='petrol_pump_unified',
    exist_ok=True,
    pretrained=True,
    
    # Optimizer settings (aligned with ProFSAM-Fire-Detector)
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=3,
    warmup_momentum=0.8,
    
    # Loss weights
    box=7.5,
    cls=0.5,
    
    # Augmentation
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.0,
    translate=0.1,
    scale=0.5,
    shear=0.0,
    flipud=0.0,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.15,
    copy_paste=0.0,
)

print("\n✅ Training completed!")

## Step 6: Evaluate Model Performance

In [ ]:
# Load best model and validate
best_model_path = MODELS_DIR / 'petrol_pump_unified' / 'weights' / 'best.pt'
model = YOLO(str(best_model_path))

# Run validation on test set
metrics = model.val(data=str(data_yaml_path), split='test')

print("\n" + "="*50)
print("MODEL PERFORMANCE")
print("="*50)
print(f"mAP50:     {metrics.box.map50:.4f}")
print(f"mAP50-95:  {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall:    {metrics.box.mr:.4f}")

In [ ]:
# Per-class performance
print("\n" + "="*50)
print("PER-CLASS PERFORMANCE")
print("="*50)
for i, class_name in enumerate(UNIFIED_CLASSES):
    if i < len(metrics.box.ap50):
        ap = metrics.box.ap50[i]
        status = "✅" if ap > 0.7 else "⚠️" if ap > 0.5 else "❌"
        print(f"{status} {class_name:12s}: AP50={ap:.4f}")
print("="*50)

## Step 7: Export for Jetson Deployment

In [ ]:
# Export to multiple formats
print("\n📦 Exporting model for deployment...")

# ONNX export (universal, works with ONNX Runtime)
onnx_path = model.export(
    format='onnx',
    imgsz=IMAGE_SIZE,
    dynamic=True,
    simplify=True
)
print(f"✅ ONNX: {onnx_path}")

# TensorRT (best done on Jetson, but try here)
try:
    engine_path = model.export(
        format='engine',
        imgsz=IMAGE_SIZE,
        half=True
    )
    print(f"✅ TensorRT: {engine_path}")
except Exception as e:
    print(f"⚠️ TensorRT skipped (export on Jetson for best results)")

In [ ]:
# Prepare output package
OUTPUT_DIR = Path('/content/petrol_pump_model')
OUTPUT_DIR.mkdir(exist_ok=True)

# Copy model files
shutil.copy(best_model_path, OUTPUT_DIR / 'petrol_pump_yolo11n.pt')

# Copy ONNX if exists
onnx_file = best_model_path.with_suffix('.onnx')
if onnx_file.exists():
    shutil.copy(onnx_file, OUTPUT_DIR / 'petrol_pump_yolo11n.onnx')

# Create labels file
with open(OUTPUT_DIR / 'labels.txt', 'w') as f:
    for i, name in enumerate(UNIFIED_CLASSES):
        f.write(f"{i}: {name}\n")

# Create usage guide
usage_guide = f"""
# Petrol Pump Unified YOLO11n Model

## Classes:
0: person     - Customer/employee detection
1: fire       - Fire/flame detection (CRITICAL)
2: smoke      - Smoke detection (early warning)
3: violence   - Fighting/violence detection
4: cigarette  - Smoking detection (CRITICAL at petrol pump)

## Usage with Ultralytics:
```python
from ultralytics import YOLO

# Load model
model = YOLO('petrol_pump_yolo11n.pt')

# Run on image
results = model.predict('image.jpg', conf=0.5)

# Run on video/RTSP stream
results = model.predict('rtsp://camera_ip:554/stream', stream=True)

# Process detections
for result in results:
    for box in result.boxes:
        class_id = int(box.cls[0])
        class_name = ['person', 'fire', 'smoke', 'violence', 'cigarette'][class_id]
        confidence = float(box.conf[0])
        
        # ALERT on critical detections
        if class_name in ['fire', 'smoke', 'cigarette']:
            print(f"🚨 ALERT: {{class_name}} detected! ({{confidence:.0%}})")
```

## Convert to TensorRT on Jetson:
```bash
yolo export model=petrol_pump_yolo11n.pt format=engine half=True
```

## Model Info:
- Base: YOLO11n
- Image Size: {IMAGE_SIZE}x{IMAGE_SIZE}
- Classes: 5
"""

with open(OUTPUT_DIR / 'README.md', 'w') as f:
    f.write(usage_guide)

print(f"\n✅ Model package ready: {OUTPUT_DIR}")
!ls -la {OUTPUT_DIR}

## Step 8: Test Inference

In [ ]:
import matplotlib.pyplot as plt
import random

# Find test images
test_images = list((MERGED_DIR / 'test' / 'images').glob('*.jpg'))
if not test_images:
    test_images = list((MERGED_DIR / 'valid' / 'images').glob('*.jpg'))

if test_images:
    # Test on multiple images
    sample_images = random.sample(test_images, min(4, len(test_images)))
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    
    for idx, img_path in enumerate(sample_images):
        results = model.predict(str(img_path), conf=0.3, verbose=False)
        result_img = results[0].plot()
        
        axes[idx].imshow(result_img[:, :, ::-1])
        axes[idx].axis('off')
        
        # Get detection summary
        detections = []
        for box in results[0].boxes:
            cls_id = int(box.cls[0])
            detections.append(UNIFIED_CLASSES[cls_id])
        axes[idx].set_title(f"Detected: {', '.join(set(detections)) if detections else 'None'}")
    
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / 'sample_detections.png', dpi=150)
    plt.show()
    print("\n✅ Sample detections saved")
else:
    print("⚠️ No test images found")

## Step 9: Download Model Package

In [ ]:
# Create downloadable zip
!cd /content && zip -r petrol_pump_yolo11n_complete.zip petrol_pump_model/

print("\n" + "="*50)
print("DOWNLOAD PACKAGE READY")
print("="*50)
print("Contents:")
print("  📦 petrol_pump_yolo11n.pt    - PyTorch model")
print("  📦 petrol_pump_yolo11n.onnx  - ONNX model")
print("  📄 labels.txt                - Class labels")
print("  📄 README.md                 - Usage guide")
print("  🖼️ sample_detections.png     - Test results")

# Download
try:
    from google.colab import files
    files.download('/content/petrol_pump_yolo11n_complete.zip')
except:
    print("\n📥 Download: /content/petrol_pump_yolo11n_complete.zip")

---

## 📊 Summary

### Model Details
| Attribute | Value |
|-----------|-------|
| Base Model | YOLO11n |
| Classes | person, fire, smoke, violence, cigarette |
| Image Size | 640x640 |
| Expected FPS (Jetson Orin) | 40-50 FPS |
| Model Size | ~6 MB |

### Data Sources
| Class | Primary Dataset | Reference |
|-------|-----------------|------------|
| person | Roboflow people-detection | COCO-style |
| fire/smoke | Roboflow fire-detection | Similar to FASDD (ProFSAM-Fire-Detector) |
| violence | Roboflow violence | Similar to fight_detection_yolov8 |
| cigarette | Roboflow smoker-detection | Custom |

### Next Steps
1. Deploy `.pt` or `.onnx` on Jetson
2. Convert to TensorRT on device for max speed
3. Fine-tune with your own petrol pump footage if needed